# Load checkpoint and imports

In [1]:
device_name='cpu'

In [2]:
import os
import sys
parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [3]:
# ! conda remove openpom
import sys
sys.path.remove('/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python39.zip')

ValueError: list.remove(x): x not in list

In [4]:
import pandas as pd
import ast
import numpy as np
# base_path  = '../../../../../../T5 EVO/alignment_olfaction_datasets/data/'
base_path = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/'

import random
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from custom_utils.data_utils import get_class_imbalance_ratio
import deepchem as dc
from utils.custom_models  import *
from constants import gs_lf_tasks
from utils.prepare_datasets import *


Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/Midgard/home/farzantn/mambaforge/envs/Mol/lib/python3.8/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [5]:
seed = 2024
np.random.seed(seed)
random.seed(seed)
# torch.cuda.manual_seed_all(seed)

# helper methods

In [6]:
# def metrics(y_true,y_predicted_prob):
#     y_pred = np.round(y_predicted_prob)
#     average_type = [None, 'macro' , 'weighted']
# 
#     # for average in average_type:
#     ap = average_precision_score(y_true, y_predicted_prob,average=None)
#     ap_macro = average_precision_score(y_true, y_predicted_prob,average='macro')
#     ap_weighted = average_precision_score(y_true, y_predicted_prob,average='weighted')
#     
#     precision = precision_score(y_true, y_pred,average=None)
#     precision_macro = precision_score(y_true, y_pred,average='macro')
#     precision_weighted = precision_score(y_true, y_pred,average='weighted')
# 
# 
#     recall = recall_score(y_true, y_pred,average=None)
#     recall_macro = recall_score(y_true, y_pred,average='macro')
#     recall_weighted = recall_score(y_true, y_pred,average='weighted')
# 
# 
#     f1 = f1_score(y_true, y_pred,average=None)
#     f1_macro = f1_score(y_true, y_pred,average='macro')
#     f1_weighted = f1_score(y_true, y_pred,average='weighted')
# 
#     
#     # true_labels = np.round(y_true)
#     
#     # print(true_labels)
#     # print(y_true)
#     return ap_macro,precision_macro,recall_macro,f1_macro
#     

In [7]:
def custom_logistic_regression(X,y):
    logreg = LogisticRegression(max_iter=1000,random_state=seed,n_jobs=-1)
    ovr = OneVsRestClassifier(logreg)
    estimator=ovr.fit(X, y)
    return estimator

In [8]:
def pipeline_regression(data_groupbyCID,times,n_components=None,seed=seed):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    for i in range(times):
        X=np.asarray(data_groupbyCID.embeddings.values.tolist())
        y=np.asarray(data_groupbyCID.y.values.tolist())
        # print("min", X.min())
        # print("max", X.max())
          
        # print("X",X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if n_components is not None:
            pca = PCA(n_components=n_components)
            X_train=pca.fit_transform(X_train)
            X_test=pca.transform(X_test)
        linreg =custom_logistic_regression(X_train,y_train)
        mseerrors, correlations=metrics_per_descritor(X_test,y_test,[str(i) for i in range(y.shape[1])],linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
    return mserrorrs_corssvalidated, correlations_corssvalidated

In [9]:
def mAP(targs, preds, pos_thr=0.5):
    """Returns the model's average precision for each class
    Return:
        ap (FloatTensor): 1xK tensor, with avg precision for each class k
    """
    def average_precision(output, target):
        epsilon = 1e-8

        # sort examples
        indices = output.argsort()[::-1]
        # Computes prec@i
        total_count_ = np.cumsum(np.ones((len(output), 1)))

        target_ = target[indices]
        ind = target_ == 1
        pos_count_ = np.cumsum(ind)
        total = pos_count_[-1]
        pos_count_[np.logical_not(ind)] = 0
        pp = pos_count_ / total_count_
        precision_at_i_ = np.sum(pp)
        precision_at_i = precision_at_i_ / (total + epsilon)

        return precision_at_i

    if np.size(preds) == 0:
        return 0
    ap = np.zeros((preds.shape[1]))
    # compute average precision for each class
    for k in range(preds.shape[1]):
        scores = preds[:, k]
        targets = targs[:, k]
        ap[k] = average_precision(scores, targets)
    tp, fp, fn, tn = [], [], [], []
    conf_matrices = []
    
    for k in range(preds.shape[0]):
        scores = preds[k,:]
        targets = targs[k,:]
        pred = (scores > pos_thr).astype(np.int32)
        tp.append(((pred + targets) == 2).sum())
        fp.append(((pred - targets) == 1).sum())
        fn.append(((pred - targets) == -1).sum())
        tn.append(((pred + targets) == 0).sum())
        conf_matrices.append(confusion_matrix(targets, pred))

    p_c = [tp[i] / (tp[i] + fp[i]) if tp[i] > 0 else 0.0 for i in range(len(tp))]
    r_c = [tp[i] / (tp[i] + fn[i]) if tp[i] > 0 else 0.0
                for i in range(len(tp))]
    f_c = [2 * p_c[i] * r_c[i] / (p_c[i] + r_c[i]) if tp[i] > 0 else 0.0
                for i in range(len(tp))]

    mean_p_c = sum(p_c) / len(p_c)
    mean_r_c = sum(r_c) / len(r_c)
    mean_f_c = sum(f_c) / len(f_c)

    p_o = sum(tp) / (np.array(tp) + np.array(fp)).sum()
    r_o = sum(tp) / (np.array(tp) + np.array(fn)).sum()
    f_o = 2 * p_o * r_o / (p_o + r_o)
    roc_auc=roc_auc_score(targs, preds)

    return ap.mean(), mean_p_c, mean_r_c, mean_f_c, p_o, r_o, f_o,roc_auc, conf_matrices


def tune_multilabel_thresholds(raw_scores, labels):
    scores = 1. / (1. + np.exp(-1. * raw_scores))
    threshs = np.zeros(labels.shape[1])

    for j in range(labels.shape[1]):
        precision, recall, thresholds = precision_recall_curve(labels[:, j], scores[:, j])
        fscore = (2 * precision * recall) / (precision + recall)
        threshs[j] = thresholds[np.argmax(fscore)]

    return threshs

In [10]:
def run_linear_classifier(modeldeepchem,embedding_dataset,embedding_test_dataset,metric,metric_str,nb_epoch = 2):
    start_time = datetime.now()
    
    train_scores=[]
    valid_scores=[]
    losses=[]
    for epoch in range(1, nb_epoch+1):
        loss = modeldeepchem.fit(
              embedding_dataset,
              nb_epoch=1,
              max_checkpoints_to_keep=1,
              deterministic=False,
              restore=epoch>1)
        losses.append(loss)
    
    modeldeepchem.save_checkpoint()
    end_time = datetime.now()
    return modeldeepchem,train_scores,valid_scores,losses


In [11]:
def create_linear_classifier(TASKS, n_classes,train_ratios,mode="classification"):
    n_classes = 1
    n_tasks= len(TASKS)
    if mode == "classification":
        model = NonLinearModel(n_tasks).to(device_name)
    elif mode == "classification_small":
        model = LinearModelSmall(n_tasks).to(device_name)
    elif mode == "linear_classification":
        model = LinearModel(n_tasks).to(device_name)
        
    # model.n_tasks=n_tasks
    model.n_classes=1
    
    # ratios = get_class_imbalance_ratio(train_dataset)
    lr = 1e-4
    
    learning_rate = 0.001
    modeldeepchem = LinearModelMultiLabelDeepchem(\
        model=model,class_imbalance_ratio=train_ratios,loss_aggr_type='sum',\
        device_name=device_name,mode=mode,optimizer_name = 'adam',\
        learning_rate=learning_rate,batch_size=128)
    
    return modeldeepchem
    

In [12]:
# def prepare_dataset(ds):
#     ds['y'] = ds['y'].apply(ast.literal_eval)
#     ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)
#     return ds

In [13]:
# def prepare_dataset_proba(ds):
#     ds['y'] = ds['y'].apply(ast.literal_eval)
#     ds['proba'] = ds['proba'].apply(ast.literal_eval)
#     return ds

In [14]:
def select_features(input_file):
    ds_alva = pd.read_csv(input_file)


    chemical_features_r=["nCIR",
                     "ZM1", 
                     "GNar", 
                     "S1K", 
                     "piPC08",
                     "MATS1v",
                     "MATS7v",
                     "GATS1v", 
                     "Eig05_AEA(bo)", 
                     "SM02_AEA(bo)",
                     "SM03_AEA(dm)",
                     "SM10_AEA(dm)",
                     "SM13_AEA(dm)",
                      "SpMin3_Bh(v)",
                     "RDF035v",
                     "G1m",
                     "G1v",
                     "G1e",
                     "G3s",
                     "R8u+",
                     "nRCOSR"]

    nonStereoSMILE = list(map(lambda x: "nonStereoSMILES___" + x, chemical_features_r))
    # IsomericSMILES = list(map(lambda x: "IsomericSMILES___" + x, chemical_features_r))
    selected_features = nonStereoSMILE
    features= ['main_idx','nonStereoSMILES']+selected_features
    # print("cc1", ds_alva.columns.values.tolist())
    ds_alva= ds_alva.rename(columns={"Unnamed: 0":"main_idx"})
    # print("cc2", ds_alva.columns.values.tolist())
    ds_alva_selected = ds_alva[features]
    ds_alva_selected = ds_alva_selected.fillna(0)
    ds_alva_selected['embeddings'] = ds_alva_selected[selected_features].values.tolist()
    return ds_alva_selected

In [15]:
def pipeline(input_file_alva,gs_lf_molformer,times=30,n_components=None,ds="keller",seed=seed):
    # input_file_keller = base_path+'openpom/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_13_Apr17.csv'
    

    
    keller_alva = select_features(input_file_keller_alva)
    keller_groupbyCID= pd.merge(keller_alva,keller_behavior,on="CID")
    
        
    mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated=pipeline_regression(keller_groupbyCID,times=times,n_components=n_components,seed=seed)
    mserrorrs_mean_keller,mserrorrs_std_keller,correlations_mean_keller,correlations_std_keller =post_process_results(mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated)
    mserrorrs_corssvalidated_keller,statistics_correlations_corssvalidated_keller,pvalues_correlations_corssvalidated_keller=post_process_results_df(mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated)
    df_keller_mse= pd.DataFrame(mserrorrs_corssvalidated_keller)
    df_keller_mse['model'] = model_name
    df_keller_cor= pd.DataFrame(statistics_correlations_corssvalidated_keller)
    df_keller_cor['model'] = model_name
    # df_keller = pd.concat((df_keller_molformer,df_keller_pom))
    return df_keller_mse, df_keller_cor

## Loading MolFormer Model

# Loading Human Embeddings

## Extracting embedding from MoLFormer for GS-LF

In [16]:
indices_train,indices_valid,indices_test = extract_set_idxs(base_path, indices_path='curated_datasets/embeddings/pom/gslf-splits-idx.csv')

In [17]:
gs_lf_molformer, gs_lf_molformer_np,gs_lf_molformer_y,gs_lf_molformer_embeddings_train,gs_lf_molformer_y_train,gs_lf_molformer_embeddings_valid,gs_lf_molformer_y_valid,gs_lf_molformer_embeddings_test,gs_lf_molformer_y_test = extract_set_from_indices(base_path, 'curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv','embeddings','y', indices_train, indices_valid, indices_test)



# input_file_molformer = base_path+'curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
# gs_lf_molformer=pd.read_csv(input_file_molformer)
# gs_lf_molformer=prepare_dataset(gs_lf_molformer,'embeddings','y')
# 
# gs_lf_molformer_np = np.asarray(gs_lf_molformer['embeddings'].tolist())
# gs_lf_molformer_y = np.asarray(gs_lf_molformer['y'].tolist())
# 
# gs_lf_molformer_embeddings_train = gs_lf_molformer_np[indices_train]
# gs_lf_molformer_y_train = gs_lf_molformer_y[indices_train]
# 
# gs_lf_molformer_embeddings_test = gs_lf_molformer_np[indices_test]
# gs_lf_molformer_y_test = gs_lf_molformer_y[indices_test]
# 
# gs_lf_molformer_embeddings_valid = gs_lf_molformer_np[indices_valid]
# gs_lf_molformer_y_valid = gs_lf_molformer_y[indices_valid]

# embedding_train_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_train,gs_lf_molformer_y_train)
# embedding_test_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_test,gs_lf_molformer_y_test)
# embedding_valid_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_valid,gs_lf_molformer_y_valid)

In [18]:

# input_file_pom = base_path + 'curated_datasets/proba/pom/gslf_pom_proba_Apr24.csv'
# gs_lf_pom=pd.read_csv(input_file_pom)
# gs_lf_pom=prepare_dataset(gs_lf_pom,'proba','y')
# 
# gs_lf_pom_np = np.asarray(gs_lf_pom['proba'].tolist())
# gs_lf_pom_y = np.asarray(gs_lf_pom['y'].tolist())
# 
# gs_lf_pom_proba_train = gs_lf_pom_np[indices_train]
# gs_lf_pom_y_train = gs_lf_pom_y[indices_train]
# 
# gs_lf_pom_proba_test = gs_lf_pom_np[indices_test]
# gs_lf_pom_y_test = gs_lf_pom_y[indices_test]
# 
# gs_lf_pom_proba_valid = gs_lf_pom_np[indices_valid]
# gs_lf_pom_y_valid = gs_lf_pom_y[indices_valid]

gs_lf_pom, gs_lf_pom_np,gs_lf_pom_y,gs_lf_pom_proba_train,gs_lf_pom_y_train,gs_lf_pom_proba_valid,gs_lf_pom_y_valid,gs_lf_pom_proba_test,gs_lf_pom_y_test = extract_set_from_indices(base_path, 'curated_datasets/proba/pom/gslf_pom_proba_Apr24.csv','proba','y', indices_train, indices_valid, indices_test)



# gs_lf_pom_embeddings_train = gs_lf_pom_np[indices_train]
# gs_lf_pom_y_train = gs_lf_pom_y[indices_train]
# 
# gs_lf_pom_embeddings_test = gs_lf_pom_np[indices_test]
# gs_lf_pom_y_test = gs_lf_pom_y[indices_test]
# 
# gs_lf_pom_embeddings_valid = gs_lf_pom_np[indices_valid]
# gs_lf_pom_y_valid = gs_lf_pom_y[indices_valid]



# embedding_train_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_train,gs_lf_pom_y_train)
# embedding_test_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_test,gs_lf_pom_y_test)
# embedding_valid_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_valid,gs_lf_pom_y_valid)

In [17]:
# gs_lf_alva.columns.values.tolist()

In [18]:
input_file_alva= base_path+'curated_datasets/alva/gslf_molecules_alva_17Apr.csv'
gs_lf_alva=select_features(input_file_alva)
# gs_lf_alva #keep embeddings

gs_lf_y=gs_lf_pom.copy() #only keep y
gs_lf_y.index.names = ['main_idx'] 
# del gs_lf_y['embeddings']
gs_lf_alva=pd.merge(gs_lf_alva, gs_lf_y, on='main_idx')
# gs_lf_alva
# gs_lf_alva.columns.values.tolist()
gs_lf_alva_np = np.asarray(gs_lf_alva.embeddings.tolist())
gs_lf_alva_y = np.asarray(gs_lf_alva.y.tolist())


gs_lf_alva_features_train = gs_lf_alva_np[indices_train]
gs_lf_alva_y_train = gs_lf_alva_y[indices_train]

gs_lf_alva_features_test = gs_lf_alva_np[indices_test]
gs_lf_alva_y_test = gs_lf_alva_y[indices_test]

gs_lf_alva_features_valid = gs_lf_alva_np[indices_valid]
gs_lf_alva_y_valid = gs_lf_alva_y[indices_valid]

# gs_lf_y.columns.

NameError: name 'base_path' is not defined

In [19]:
embedding_train_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_train,gs_lf_alva_y_train)
embedding_test_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_test,gs_lf_alva_y_test)
embedding_valid_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_valid,gs_lf_alva_y_valid)

NameError: name 'dc' is not defined

# Training a Linear Classifier using DeepChem 

##  Training a non-Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [20]:
mAP(gs_lf_pom_y_test, gs_lf_pom_proba_test, pos_thr=0.5)

NameError: name 'gs_lf_pom_y_test' is not defined

### molformer

In [21]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_pom)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf_pom,embedding_test_dataset_gslf_pom,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [22]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### Alva

In [23]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [24]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### POM

In [25]:
mAP(gs_lf_pom_y_test, gs_lf_pom_proba_test, pos_thr=0.5)

NameError: name 'gs_lf_pom_y_test' is not defined

### molformer

In [26]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios)
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [27]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### Alva

In [28]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [29]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

In [30]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [31]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [32]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [33]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [34]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_pom_y_test' is not defined

### molformer

In [35]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [36]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### Alva

In [37]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [38]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### Alva

In [39]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [40]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

In [41]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [42]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [43]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [44]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [45]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_pom_y_test' is not defined

### molformer

In [46]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [47]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

### Alva

In [48]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [49]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [50]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined

In [51]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [52]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [53]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [54]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [55]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_pom_y_test' is not defined

### molformer

In [56]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

NameError: name 'get_class_imbalance_ratio' is not defined

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [57]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

NameError: name 'get_class_imbalance_ratio' is not defined

In [58]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

NameError: name 'gs_lf_molformer_y_test' is not defined